In [2]:
import numpy as np
import qutip as qt
from qutip import identity, sigmax, sigmay, sigmaz, tensor
import matplotlib.pyplot as plt
import tensorflow as tf
from matplotlib import rc
from matplotlib import cm
import matplotlib as mpl
rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
rc('font',**{'family':'serif','serif':['Times']})
rc('text', usetex=True)
import timeit
import time



2023-03-08 12:41:29.912953: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [22]:
def generate_random_L(H_dim):
    L = tf.cast(tf.complex(np.random.uniform(-1,1,size=(H_dim,H_dim)),
                               np.random.uniform(-1,1,size=(H_dim,H_dim))),
                    dtype=tf.complex64)
    trace_L = tf.linalg.trace(L)
    traceless_L = L - (trace_L/H_dim)*tf.eye(num_rows=H_dim, dtype=tf.complex64)
    return traceless_L

def build_rho(Vars):
    # Generate a random density matrix by using a cholesky composition and normalising
    
    # separate diagonal elements
    diag_elems = tf.linalg.diag_part(Vars)
    # make a matrix with these on the diag
    diag_matrix = tf.linalg.diag(diag_elems) #+ tf.linalg.diag([0.001] * Var.shape[0])
    # build a lower triangular form with real diagonals
    real_lower_tri = (tf.linalg.LinearOperatorLowerTriangular(Vars).to_dense() - diag_matrix + tf.abs(diag_matrix))
    # get the upper triangular half with zeros on the diagonal, to act as the complex part
    im_lower_tri = (tf.linalg.LinearOperatorLowerTriangular(tf.transpose(Vars, perm=[1,0])).to_dense() - diag_matrix)  
    # build L and L_dag
    L = tf.complex(real=real_lower_tri, imag=im_lower_tri)
    L_dag = tf.linalg.adjoint(L)
    # get A
    A = tf.cast(tf.matmul(L,L_dag), dtype=tf.complex64)
    norm = tf.linalg.trace(A)
    A_normed = A/norm
    return A_normed

def calculate_output(jump, targ_state, gamma=1):
        output = gamma*(tf.matmul(tf.matmul(jump,targ_state),jump, adjoint_b=True) -
                   (1/2)*(tf.matmul(tf.matmul(jump,jump, adjoint_a=True),targ_state)
                         + tf.matmul(targ_state,tf.matmul(jump,jump, adjoint_a=True))))
        dist = tf.reduce_sum(tf.abs(output)) 
        return dist
    
# real_elems = np.loadtxt("Best_GKP_jump_200_real_elems")
# imag_elems = np.loadtxt("Best_GKP_jump_200_imag_elems")
# best_jump = real_elems +1j*imag_elems

In [54]:
def find_steady_state(H_dim, jump_op, scheduler):
    Vars = tf.Variable(np.random.uniform(-1,1,size=(H_dim, H_dim)))
    optim_1 = tf.keras.optimizers.Adam(learning_rate=scheduler,
                                      amsgrad=True)
    cost_ls = []
    steps=0
    done=False
    benchmark = 10000
    cost_ls.append(benchmark)
    while not done:
        with tf.GradientTape() as tape:
            tape.watch(Vars)
            rho = build_rho(Vars)
            dist = calculate_output(jump_op, rho) 
            cost_ls.append(dist) 

        grads = tape.gradient(dist, Vars)
        optim_1.apply_gradients(zip([grads], [Vars]));
        print("Step: {}, Dist: {}".format(steps, dist), end='\r')
        steps+=1
        if dist <benchmark:
            best_rho = rho
            best_vars = Vars
            benchmark = dist
            if benchmark<0.01:
                done=True
        if steps==2000:
            done=True
    return best_rho



In [73]:
scheduler = tf.keras.optimizers.schedules.ExponentialDecay(0.1, 
                                                          decay_steps=50, 
                                                          decay_rate=0.98)
H_dims = np.arange(1,12)*5
num_runs = 10
qutip_times = np.zeros((12,num_runs))
gd_times = np.zeros((12,num_runs))
gd_fidelities = np.zeros((12,num_runs))
for i, H_dim in enumerate(H_dims):
    for j in range(num_runs):
        jump = generate_random_L(H_dim = H_dim)
        L = qt.Qobj(jump.numpy())
        Liouv = (qt.sprepost(L,L.dag()) - 
                 (1/2)*(qt.spre(L.dag()*L) + qt.spost(L.dag()*L)))
        a = time.time()
        true_ss = qt.steadystate(Liouv, method='eigen')
        b = time.time()
        qutip_times[i,j] = b-a
        c = time.time()
        best_rho = find_steady_state(H_dim=H_dim, jump_op=jump, scheduler = scheduler)
        d = time.time()
        gd_times[i,j] = d-c
        best_guess= qt.Qobj(best_rho.numpy())
        gd_fidelities[i,j] = qt.fidelity(best_guess, true_ss)

In [74]:
np.savetxt("qt_times", qutip_times)
np.savetxt("gd_times", gd_times)
np.savetxt("gd_fidelities", gd_fidelities)

# fig, ax = plt.subplots(1,1)
# extent = np.linspace(-5,5,200)
# W = qt.distributions.wigner(normed_best_guess, xvec=extent, yvec=extent)
# # qt.visualization.plot_wigner(targ_state_qt, colorbar=True, figsize = (5,5), alpha_max=4);
# wlim = abs(W).max()
# cmap = cm.get_cmap("RdBu")
# cf = ax.contourf(extent, extent, W, 100,
#                          norm=mpl.colors.Normalize(-wlim, wlim), cmap=cmap)
# # plt.imshow(W, cmap=cf, origin="lower")
# plt.colorbar(cf, ax=ax)
# # plt.title(r"$|0_L \rangle$", fontsize=20)

# plt.ylabel(r"$\beta$", fontsize=15)
# plt.xlabel(r"$\alpha$", fontsize=15)